# Instalación de librerias

Lo primero que debemos hacer es instalar las librerias de Beatiful Soup y requests.

In [1]:
!pip3 install beautifulsoup4 --upgrade
!pip3 install requests
!pip3 install selenium
!pip install webdriver-manager

# Importación de librerias

Vamos a importar las librerias que tenemos que utilizar.

In [2]:
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import json
import pandas as pd
import requests
from os.path  import basename
import time
from IPython.display import clear_output
from multiprocessing.pool import ThreadPool as Pool

# Obtención del listado de Cripto Monedas

Lo primero que vamos a obtener es el listado de las cripto monedas listadas en coinmarketcap. Para ello haremos el analisis de la home donde podemos obtener este listado y algunos destalles de dichas monedas. 

Obtenemos una versión mas facil de trabajar de nuestra pagina web.


In [3]:
#Utilizamos la url donde podemos listar todas las criptos

url = "https://coinmarketcap.com/all/views/all/"

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument("--disable-extensions")


driver = webdriver.Chrome(executable_path=ChromeDriverManager().install(), options=chrome_options)
driver.maximize_window()
driver.get(url)
html = driver.page_source.encode('utf-8')
page_num = 0
time.sleep(5)
try:
    #Mientras que el exita
    while driver.find_element_by_xpath('//button[contains(text(), "Load More")]'):
        driver.find_element_by_xpath('//button[contains(text(), "Load More")]').click()
        page_num += 1
        print("getting page number "+str(page_num))
        time.sleep(3)
except:
    print("Se ha finalizado la carga de paginas")
    
html = driver.page_source.encode('utf-8')

driver.close() # Cerramos el nevegador.

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 95.0.4638
[WDM] - Get LATEST driver version for 95.0.4638
[WDM] - Driver [C:\Users\Carlos\.wdm\drivers\chromedriver\win32\95.0.4638.17\chromedriver.exe] found in cache
<ipython-input-3-ccb1692a799b>:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=ChromeDriverManager().install(), options=chrome_options)
<ipython-input-3-ccb1692a799b>:18: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  while driver.find_element_by_xpath('//button[contains(text(), "Load More")]'):
<ipython-input-3-ccb1692a799b>:19: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('//button[contains(text(), "Load More")]').click()


Se ha finalizado la carga de paginas


In [4]:
soup = BeautifulSoup(html, "html.parser")
print(soup.title)

<title>All Cryptocurrencies | CoinMarketCap</title>


In [5]:
print(soup.prettify())

<html lang="en">
 <head>
  <script async="" src="https://www.google-analytics.com/plugins/ua/linkid.js" type="text/javascript">
  </script>
  <script async="" src="https://www.google-analytics.com/analytics.js" type="text/javascript">
  </script>
  <script async="" src="//static.ads-twitter.com/uwt.js">
  </script>
  <script async="" src="https://connect.facebook.net/signals/config/2401726993442574?v=2.9.47&amp;r=stable">
  </script>
  <script async="" src="https://connect.facebook.net/signals/config/252996273495174?v=2.9.47&amp;r=stable">
  </script>
  <script defer="" src="https://connect.facebook.net/en_US/fbevents.js">
  </script>
  <script async="" src="//www.googleadservices.com/pagead/conversion_async.js" type="text/javascript">
  </script>
  <script async="" src="https://static.hotjar.com/c/hotjar-1060636.js?sv=7" type="text/javascript">
  </script>
  <script async="" charset="UTF-8" src="https://cdn.jsdelivr.net/npm/sa-sdk-javascript@1.15.26/sensorsdata.min.js">
  </script>
  

In [6]:
data=[]
links = soup.find_all('a', class_='cmc-table__column-name--name cmc-link')
links2 = soup.find_all('a', class_='cmc-link')


for row in links:
    link = row['href']
    data.append((link))
    
for link in links2:
    if link['href'].startswith('/currencies/') and not(link['href'].endswith('/markets/')):
        if link['href'] not in data:
            data.append(link['href'])
            
print(len(data))
    
for item in data:
    print(item)


200
/currencies/bitcoin/
/currencies/ethereum/
/currencies/binance-coin/
/currencies/cardano/
/currencies/tether/
/currencies/solana/
/currencies/xrp/
/currencies/polkadot-new/
/currencies/dogecoin/
/currencies/usd-coin/
/currencies/terra-luna/
/currencies/shiba-inu/
/currencies/uniswap/
/currencies/chainlink/
/currencies/avalanche/
/currencies/wrapped-bitcoin/
/currencies/litecoin/
/currencies/binance-usd/
/currencies/algorand/
/currencies/bitcoin-cash/
/currencies/polygon/
/currencies/band-protocol/
/currencies/prometeus/
/currencies/status/
/currencies/orchid/
/currencies/velas/
/currencies/cartesi/
/currencies/energy-web-token/
/currencies/nkn/
/currencies/badger-dao/
/currencies/husd/
/currencies/origin-protocol/
/currencies/constellation/
/currencies/oasis-network/
/currencies/myneighboralice/
/currencies/orbs/
/currencies/ergo/
/currencies/stratis/
/currencies/ark/
/currencies/augur/
/currencies/casper/
/currencies/enzyme/
/currencies/maidsafecoin/
/currencies/steem/
/currencies

# Scraping de las páginas de cada critpomoneda.
Una vez obtenido el listado de monedas, recorremos cada una de ellas y accedemos a sus precios hitoricos que seran los que almacenemos en nuestro dataset.


In [7]:
dataName=[]
dataSymbol=[]
dataLogo=[]
dataUrl=[]
dataHistoricFile=[]
dataHistoricDate=[]
dataHistoricOpen=[]
dataHistoricHigh=[]
dataHistoricLow=[]
dataHistoricClose=[]
dataHistoricVolume=[]
dataHistoricMarketCap=[]
dataHistoricSymbol=[]
i=0

try:
    driver.close()
except:
    pass
        

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument("--disable-extensions")


driver = webdriver.Chrome(executable_path=ChromeDriverManager().install(), options=chrome_options)

for item in data:
#for item in data[:2500]:
    #dataHistoricDate=[]
    #dataHistoricOpen=[]
    #dataHistoricHigh=[]
    #dataHistoricLow=[]
    #dataHistoricClose=[]
    #dataHistoricVolume=[]
    #dataHistoricMarketCap=[]
    #dataHistoricSymbol=[]
    i=i+1
    print(i)
    try:
        url = "https://coinmarketcap.com" + item +  "historical-data/"
        driver.maximize_window()
        driver.get(url)
        
        time.sleep(1)
        html = driver.page_source.encode('utf-8')
        
        
        #Seleccionamos con Selenium el historico de 365 dias
        driver.find_element_by_xpath('//Button[contains(text(), "Date Range")]').click() 
        driver.find_element_by_xpath('//li[contains(text(), "Last 365 days")]').click()
        driver.find_element_by_xpath('//button[contains(text(), "Continue")]').click()
        
        
        time.sleep(1)
        html = driver.page_source.encode('utf-8')


        soup = BeautifulSoup(html, "html.parser")

        table = soup.find('table').find('tbody')    
        
        #Parseamos logo y nombre y codigo de la crypto.
        divsTitle = soup.find_all('div', class_='sc-16r8icm-0 gpRPnR nameHeader')
        
        url = item
        dataUrl.append(url)
        
        symbol= divsTitle[0].small.text
        dataSymbol.append(symbol)
    
        name= divsTitle[0].h2.text.replace(symbol,'')
        dataName.append(name)
    
        logoURL = divsTitle[0].img.get('src')
        
        
        dataLogo.append(symbol+".png")
    
        dataHistoricFile.append(symbol+'.csv')
        
        #Parseamos tabla de cotizaciones.
        for tr in table.find_all('tr'):
            tds = tr.find_all('td')
            dataHistoricSymbol.append(symbol)
            dataHistoricDate.append(tds[0].text)
            dataHistoricOpen.append(tds[1].text)
            dataHistoricHigh.append(tds[2].text)
            dataHistoricLow.append(tds[3].text)
            dataHistoricClose.append(tds[4].text)
            dataHistoricVolume.append(tds[5].text)
            dataHistoricMarketCap.append(tds[6].text)
        
        #dictHistoric={'symbol': dataHistoricSymbol, 'date':dataHistoricDate, 'open': dataHistoricOpen,
        #     'high': dataHistoricHigh, 'low':dataHistoricLow, 'close':dataHistoricClose, 'volume':dataHistoricVolume,
        #     'marketCap':dataHistoricMarketCap}
        #df_hist = pd.DataFrame(dictHistoric)

        #df_hist.to_csv("./historicData/"+symbol+".csv", index=False)
        
        
        
        with open("./images/"+symbol+".png","wb") as f:
                f.write(requests.get(logoURL).content)
        
    
        
        clear_output(wait=True)
       
    except:
        pass

    
dictHistoric={'symbol': dataHistoricSymbol, 'date':dataHistoricDate, 'open': dataHistoricOpen,
              'high': dataHistoricHigh, 'low':dataHistoricLow, 'close':dataHistoricClose, 'volume':dataHistoricVolume,
              'marketCap':dataHistoricMarketCap}
df_hist = pd.DataFrame(dictHistoric)
df_hist.to_csv("total.csv", index=False)
    
dictData={'name': dataName, 'symbol': dataSymbol, 'logo': dataLogo, 'historicalFile': dataHistoricFile, 'url':dataUrl}
df = pd.DataFrame(dictData)
df.to_csv("coin_info.csv",index=False)     
driver.close()

200


In [ ]:
dataHistoricOpen

In [ ]:
def updateHist(ruta,symbol):
    print("entra " + symbol)
    dataName=[]
    dataSymbol=[]
    dataLogo=[]
    dataUrl=[]
    dataHistoricDate=[]
    dataHistoricOpen=[]
    dataHistoricHigh=[]
    dataHistoricLow=[]
    dataHistoricClose=[]
    dataHistoricVolume=[]
    dataHistoricMarketCap=[]
    dataHistoricSymbol=[]
    clear_output(wait=True)
    try:
        df = pd.read_csv("./historicData/"+symbol+'.csv')  
        
        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_argument('--no-sandbox')
        chrome_options.add_argument("--disable-extensions")

        driver = webdriver.Chrome(executable_path=ChromeDriverManager().install(), options=chrome_options)    


        url = "https://coinmarketcap.com" + ruta +  "historical-data/"
        driver.maximize_window()
        driver.get(url)
        
        time.sleep(1)
        html = driver.page_source.encode('utf-8')
    
        soup = BeautifulSoup(html, "html.parser")
        table = soup.find('table').find('tbody')
        
        dataSymbol.append(symbol)
        
        #dataHistoricFile.append(symbol+'.csv')
                #Parseamos tabla de cotizaciones.
        for tr in table.find_all('tr'):
            tds = tr.find_all('td')
            dataHistoricSymbol.append(symbol)
            dataHistoricDate.append(tds[0].text)
            dataHistoricOpen.append(tds[1].text)
            dataHistoricHigh.append(tds[2].text)
            dataHistoricLow.append(tds[3].text)
            dataHistoricClose.append(tds[4].text)
            dataHistoricVolume.append(tds[5].text)
            dataHistoricMarketCap.append(tds[6].text)
        
        dictHistoric={'symbol': dataHistoricSymbol, 'date':dataHistoricDate, 'open': dataHistoricOpen,
                     'high': dataHistoricHigh, 'low':dataHistoricLow, 'close':dataHistoricClose, 'volume':dataHistoricVolume,
                     'marketCap':dataHistoricMarketCap}
        df_hist = pd.DataFrame(dictHistoric)
        df.reset_index(drop=True, inplace=True)
        df_hist.reset_index(drop=True, inplace=True)
        
        df_end = pd.concat([df_hist,df], axis=0).drop_duplicates(subset=['date']).reset_index(drop=True)
        
        df_end.to_csv("./historicData/"+symbol+".csv", index=False)
        driver.close()
    except:
        driver.close()



In [ ]:
horaIncio = time.strftime("%c")
pool_size = 7
pool = Pool(pool_size)
df = pd.read_csv("coin_info.csv")  
for index, row in df.iterrows():
    pool.apply_async(updateHist, (row["url"],row["symbol"],))
    #updateHist(row["url"],row["symbol"],driver)

pool.close()
pool.join()

print("Fecha y hora Inicio " + horaIncio)
print ("Fecha y hora Fin " + time.strftime("%c"))

In [ ]:
updateHist("/currencies/bitcoin/","BTC")